# Mini-lab 3 - Maps as a visualization tool

In [1]:
from datascience import *
import numpy as np
from geopy.geocoders import Nominatim

### Geocoding
Geocoding is the process of converting locations (such as address or points of interest) into geographic coordinates (such as lat, lon coordinates). The geopy.geocoders package gives us access to several geolocation API's (like google maps, Nominatim - open street maps API, bing maps API, etc.). The documentations for the geocoding package can be found here: https://geopy.readthedocs.io/en/1.10.0/. 


#### Geocoding example
Below is an example of how the geocoding service works:

In [2]:
geolocator = Nominatim()
location = geolocator.geocode("175 5th Avenue NYC")
location


Location(Flatiron Building, 175, 5th Avenue, Flatiron, Manhattan, New York County, NYC, New York, 10010, United States of America, (40.7410861, -73.9896297241625, 0.0))

#### location properties

In [3]:
location.latitude, location.longitude, location.altitude, location.address,

(40.7410861,
 -73.9896297241625,
 0.0,
 'Flatiron Building, 175, 5th Avenue, Flatiron, Manhattan, New York County, NYC, New York, 10010, United States of America')

#### Use the geolocater to find the lat lon coordinates for the Statue of Liberty

In [4]:
# Your code here
geolocator = Nominatim()
location = geolocator.geocode("Statue of Liberty")
location



Location(Statue of Liberty, Hudson River Waterfront Walkway, Jersey City, Hudson County, New Jersey, 07302, United States of America, (40.6892474, -74.0445404280149, 0.0))

# Creating maps
If we generate a table with columns 'latitude', 'longitude', 'popup', and 'color'. We can generate maps with  markers from the table using the maps.Marker.map_table() method. See the example below. Click on the marker and the popup text should appear. 

In [3]:
location = geolocator.geocode("175 5th Avenue NYC")
table = Table().with_columns(['latitude', [location.latitude],
                              'longitude', [location.longitude],
                              'popup', [location.address],
                              'color', ['red']])
table

latitude,longitude,popup,color
40.7411,-73.9896,"Flatiron Building, 175, 5th Avenue, Flatiron, Manhattan, ...",red


In [4]:
table['popup']=str(table['latitude'])

In [6]:
maps.Marker.map_table(table)

### Add a row to the table for the empire state building. Make the color blue
The [Table.append()](http://data8.org/datascience/_autosummary/datascience.tables.Table.append.html?highlight=append#datascience.tables.Table.append) method may come in handy for adding a row.

In [5]:
# Your code here
location = geolocator.geocode("Statue of Liberty")
table.append([location.latitude, location.longitude, location.address, 'green'])
table


latitude,longitude,popup,color
40.7411,-73.9896,[ 40.7410861],red
40.6892,-74.0445,"Statue of Liberty, Hudson River Waterfront Walkway, Jers ...",green


In [18]:
table['popup'] = table.apply(lambda x: str(x), 'latitude')

table

'[ 40.7410861  40.6892474]'

In [16]:
maps.Marker.map_table(table)

In [17]:
# If we add a radius column, we can use circle markers instead of info markers
table['radius'] = 200
maps.Circle.map_table(table)

### Mapping utilities and services in San Francisco
Mapping can be an important tool for visualizing spatial data. In the rest of this lab we will try to get more familiar with the mapping tools.

#### Mapping firestations in SF
'firestations.csv' is a list of addresses of firestations in San Francisco. 

**Your tasks:** 
1. Use the geolocater to find the lat, lon coordinates of each address, **I recommend adding ', San Francisco, CA' to each address before trying to geolocate it.** Add the latitude and longitude info to the firestations table. 
2. Create a table that contains the columns ‘latitude’, ‘longitude’, ‘popup’, ‘color’, ‘radius’ with color = 'red', radius = 200, and popup = the Station # (from the firestation table).
3. Use maps.Circle.map_table(table) to create a map of the points


TODO: use for loop.

In [3]:
firestations = Table.read_table('minilab3/firestations.csv')
firestations

Station #,Address
Station 1,935 Folsom Street
Station 2,1340 Powell Street
Station 3,1067 Post Street
Station 4,449 Mission Rock Street
Station 5,1301 Turk Street
Station 6,135 Sanchez Street
Station 7,2300 Folsom Street
Station 8,36 Bluxome Street
Station 9,2245 Jerrold Avenue
Station 10,655 Presidio Avenue


In [4]:
fire_stations_map_table = Table(['latitude', 'longitude', 'popup', 'color', 'radius'])

i = 0
for address in firestations['Address'][0:10]:
    station = firestations['Station #'][i]
    location = geolocator.geocode(address+', San Francisco, CA')
    fire_stations_map_table.append([location.latitude, location.longitude, station, 'black', 200])
    i = i+1




In [5]:

maps.Circle.map_table(fire_stations_map_table)


In [6]:
from geopy.distance import great_circle
location = geolocator.geocode("San Francisco City Hall")
lat1, lon1 = location.latitude, location.longitude
dists = fire_stations_map_table.apply(lambda lat2, lon2 :
                                      great_circle((lat1, lon1),
                                                   (lat2,lon2)).miles, 
                                      ['latitude','longitude'])
fire_stations_map_table = fire_stations_map_table.append([lat1, lon1, 'City hall', 'blue', 300])


In [7]:
np.argsort(dists)[0:3]

array([2, 4, 0])

In [8]:
fire_stations_map_table['color'][np.argsort(dists)[0:3]] = 'red'
maps.Circle.map_table(fire_stations_map_table)

In [33]:
fire_stations_map_table

latitude,longitude,popup,color,radius
37.7793,-122.404,Station 1,black,200
37.797,-122.41,Station 2,black,200
37.7866,-122.419,Station 3,red,200
37.7728,-122.389,Station 4,black,200
37.7805,-122.431,Station 5,red,200
37.7671,-122.431,Station 6,black,200
37.7601,-122.415,Station 7,black,200
37.7773,-122.397,Station 8,black,200
37.7452,-122.401,Station 9,black,200
37.7856,-122.447,Station 10,black,200
